In [ ]:
!pip install git+https://github.com/qubvel/segmentation_models.pytorch
!pip install adabelief-pytorch==0.2.0
!pip install monai
!pip install torchvision

!jupyter nbextension enable --py widgetsnbextension
!pip install ipywidgets widgetsnbextension

In [ ]:
import json
import numpy as np

import PIL.Image
import PIL.ImageDraw
from PIL import Image

import matplotlib.pyplot as plt
import matplotlib

import os
import cv2

# Define shape_to_mask function


In [ ]:
def shape_to_mask(
    img_shape, points, shape_type=None, line_width=10, point_size=5
):
    mask = np.zeros(img_shape[:2], dtype=np.uint8) 
    mask = PIL.Image.fromarray(mask) 
    
    draw = PIL.ImageDraw.Draw(mask) 

    xy = [tuple(point) for point in points] 
    
    
    if shape_type == "circle":
        assert len(xy) == 2, "Shape of shape_type=circle must have 2 points"
        (cx, cy), (px, py) = xy
        d = math.sqrt((cx - px) ** 2 + (cy - py) ** 2)
        draw.ellipse([cx - d, cy - d, cx + d, cy + d], outline=1, fill=1) 
    elif shape_type == "rectangle":
        assert len(xy) == 2, "Shape of shape_type=rectangle must have 2 points"
        draw.rectangle(xy, outline=1, fill=1)
    elif shape_type == "line":
        assert len(xy) == 2, "Shape of shape_type=line must have 2 points"
        draw.line(xy=xy, fill=1, width=line_width)
    elif shape_type == "linestrip":
        draw.line(xy=xy, fill=1, width=line_width)
    elif shape_type == "point":
        assert len(xy) == 1, "Shape of shape_type=point must have 1 points"
        cx, cy = xy[0]
        r = point_size
        draw.ellipse([cx - r, cy - r, cx + r, cy + r], outline=1, fill=1)
    else:
        assert len(xy) > 2, "Polygon must have points more than 2" 
        draw.polygon(xy=xy, outline=1, fill=1)
    mask = np.array(mask, dtype=bool)
    return mask 

# Set process folder

In [ ]:
# path
folder_path = "SEG_Train_Datasets/Train_Annotations/"

# Create Visualize Function

In [ ]:
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 16)) 
    for i, (name, image) in enumerate(images.items()):   
        plt.subplot(1, n, i + 1)  
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()

# Processing all json file in the folder (remember to create a "msk_img" folder for storing the results)


In [ ]:
try: 
    os.makedirs(folder_path.replace("Train_Annotations", "Train_Annotations_png"))
except: 
    pass


for filename in os.listdir(folder_path):
    if 'json' in filename:
        # Read in all all the data from the CSV file  
        json_path = os.path.join(folder_path, filename)       
        
        write_msk_img_name = filename.replace("json","png")
        write_folder_path = folder_path.replace("Train_Annotations","Train_Annotations_png")
        
        img_path = folder_path.replace("/Train_Annotations/","/Train_Images/") + filename.replace("json","jpg")
        img = Image.open(img_path)

        #Read Json file
        with open(json_path, "r",encoding="utf-8") as f:
            dj = json.load(f)

        
        temp_mask_img = np.zeros([dj['imageHeight'], dj['imageWidth']],dtype=np.uint8)

        #Plot each mask into mask_img
        for i in range(len(dj['shapes'])):
            mask = shape_to_mask((dj['imageHeight'],dj['imageWidth']), dj['shapes'][i]['points'], shape_type=dj['shapes'][i]['shape_type'],line_width=1, point_size=1)            
            temp_mask_img = temp_mask_img + mask.astype(int) 
        temp_mask_img = (temp_mask_img>0).astype(int)
        
        if int(filename.split('.')[0])%300 == 0:
            visualize(
                image = img,
                mask = temp_mask_img
                )
        print(f"temp_mask_img.range is {temp_mask_img.max()} to {temp_mask_img.min()}")
        # Save the file
        cv2.imwrite(write_folder_path + write_msk_img_name, temp_mask_img*255)

# Import Packages

1. [logging](https://docs.python.org/zh-tw/3/howto/logging.html)

In [ ]:
!nvidia-smi

In [ ]:
import logging
import os
import sys
import tempfile
import glob
import time
import matplotlib.pyplot as plt
import numpy as np

import setuptools
import torch
import torchvision
from PIL import Image
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import monai
from monai.data import create_test_image_2d, list_data_collate, decollate_batch
from monai.inferers import sliding_window_inference, SimpleInferer
from monai.metrics import DiceMetric
from monai.transforms import (
    Activations,
    AddChanneld,
    AsDiscrete,
    Compose,  
    LoadImaged,
    RandCropByPosNegLabeld,
    RandRotate90d,
    ScaleIntensityd,
    EnsureTyped,
    EnsureType,
    AsChannelFirstd,
    AsChannelLast,
    Resized,
#     RandScaleCropd,
    RandRotated,
    SaveImage,
#     RandSpatialCropd,
    Resize,
    RandFlipd,
#     RandRotate,
)
from monai.visualize import plot_2d_or_3d_image

# Check MONAI configurations

In [ ]:
monai.config.print_config()
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# Process VGH Data

In [ ]:
# Set the Data folder
data_path = './SEG_Train_Datasets/'
os.listdir(data_path)

In [ ]:
# Load train files
for_test = 253


tempdir = os.path.join(data_path, "Train_Images")
train_images = sorted(glob.glob(os.path.join(tempdir, "*.jpg")))

tempdir = os.path.join(data_path, "Train_Annotations_png")
train_segs = sorted(glob.glob(os.path.join(tempdir, "*.png")))


print(f" {len(train_images[for_test:])} train_images and {len(train_segs[for_test:])} train_segs")


train_files = [{"img": img, "seg": seg} for img, seg in zip(train_images[for_test:], train_segs[for_test:])]



# Data divided into Train and Validation

In [ ]:
val_files = [{"img": img, "seg": seg} for img, seg in zip(train_images[:for_test], train_segs[:for_test])]
print(f" {len(train_images[:for_test])} val_images and {len(train_segs[:for_test])} val_segs")

# Define Trasform for image and segmentation



In [ ]:
# define transforms for image and segmentation
train_transforms = Compose(
    [
        LoadImaged(keys=["img", "seg"]),
        AddChanneld(keys=["seg"]),        
        AsChannelFirstd(keys=["img"]),
        ScaleIntensityd(keys=["img", "seg"]),
        RandFlipd(keys=["img", "seg"], prob=0.3),
        RandRotated(keys=["img", "seg"],range_x= 5),
        Resized(keys=["img", "seg"], spatial_size=[800, 800]),
        EnsureTyped(keys=["img", "seg"]),
    ]
)
val_transforms = Compose(
    [
        LoadImaged(keys=["img", "seg"]),
        AddChanneld(keys=["seg"]),        
        AsChannelFirstd(keys=["img"]),
        ScaleIntensityd(keys=["img", "seg"]),
        Resized(keys=["img"], spatial_size=[800, 800]),
        EnsureTyped(keys=["img", "seg"]),
    ]
)

# Create DataLoader for train and validation data


In [ ]:
# batch_size 
batch_size = 16
# create a training data loader
train_ds = monai.data.Dataset(data=train_files, transform=train_transforms)
train_loader = DataLoader(
    train_ds,
    batch_size= batch_size,
    shuffle=True,
    collate_fn=list_data_collate,
    pin_memory=torch.cuda.is_available(),
)

# create a validation data loader
val_ds = monai.data.Dataset(data=val_files, transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size= batch_size, collate_fn=list_data_collate)



# Define metric and post-processing


In [ ]:
dice_metric = DiceMetric(include_background=False, reduction="mean", get_not_nans=False)
post_trans = Compose([EnsureType(), Activations(sigmoid=True), AsDiscrete(threshold=0.5)])

# Built Model



In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0, 1, 2, 3'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


# Create Visualize Function


In [ ]:
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 16))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image, cmap= 'gray')
    plt.show()

# Model Training



[Open TensorBoard](/tensorboard/)

In [ ]:
import segmentation_models_pytorch as smp

In [ ]:
encode_mod = 'EfficientV2'

In [ ]:
# smp官方資訊
aux_params=dict(
    pooling='avg',             # one of 'avg', 'max'
    dropout=0.4,               # dropout ratio, default is None
    activation=None,           # activation function, default is None
    classes=1,                 # define number of output labels
)


encoder_type = 'tu-tf_efficientnetv2_m_in21ft1k'
model_no = smp.DeepLabV3Plus(encoder_type, aux_params=aux_params)

In [ ]:
model = torch.nn.DataParallel(model_no).to(device)

In [ ]:
#AdaBelief
from adabelief_pytorch import AdaBelief
optimizer = AdaBelief(model.parameters(), lr=1.5e-4, eps=1e-16, betas=(0.9, 0.99), weight_decouple = True, rectify = False, weight_decay = 1e-4)
loss_function = monai.losses.DiceLoss(sigmoid=True)

In [ ]:
!nvidia-smi

In [ ]:
#### start a typical PyTorch training
total_epochs = 500
val_interval = 1
best_metric = 100  
best_metric_epoch = -1 
epoch_loss_values = list()  
metric_values = list()
writer = SummaryWriter()  
lr_set = []

for epoch in range(total_epochs):
    print(f"Epochs {epoch + 1}/{total_epochs}")

    model.train()  
    epoch_loss = 0
    step = 0

    for i, batch_data in enumerate(train_loader):

        step += 1
        epoch_len = len(train_ds) // train_loader.batch_size   #epoch數
        print(f"Step = {step}/{epoch_len}", end='\r')

        inputs, labels = batch_data["img"].to(device), batch_data["seg"].to(device)
        optimizer.zero_grad()
        
        outputs, _ = model(inputs) #forward
        
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += float(loss.item())

        writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)


    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    train_loss.append(epoch_loss)

    if (epoch + 1) % val_interval == 0:
        model.eval()
        with torch.no_grad():
            val_images = None
            val_labels = None
            val_outputs = None
            steps = 0
            loss_val = 0

            for val_data in val_loader:
                val_images, val_labels = val_data["img"].to(device), val_data["seg"].to(device)
                
                val_outputs, _ = model(val_images) #forward
                val_outputs = Resize([-1, 1716, 942])(val_outputs)

                val_loss = monai.losses.DiceLoss(sigmoid=True)(val_outputs, val_labels)
                loss_val += float(val_loss)

                # val 
                val_outputs = [post_trans(i) for i in decollate_batch(val_outputs)]                

                # compute metric for current iteration
                if  steps < 5:
                    print("validation loss = ", val_loss)
                    visualize( 
                        image=val_images[0].cpu().permute(1,2,0), 
                        ground_truth_mask=val_labels[0].cpu().permute(1,2,0), 
                        predicted_mask=val_outputs[0].cpu().permute(1,2,0)

                    )  
                steps += 1
                dice_metric(y_pred=val_outputs, y=val_labels)


            # aggregate the final mean dice result
            print("val_loss = ", loss_val / steps)
            val_loss_ave = loss_val / steps

            # reset the status for next validation round
            dice_metric.reset()
            metric_values.append(val_loss_ave)
            test_loss.append(val_loss_ave)

            if val_loss_ave < best_metric:
                best_metric = val_loss_ave
                best_metric_epoch = epoch + 1
                torch.save(model.state_dict(), f"{encode_mod}.pth")
                print("saved new best metric model")
            print(
                "current epoch: {} current val mean dice loss: {:.4f} best val mean dice loss: {:.4f} at epoch {}".format(
                    epoch + 1, val_loss_ave, best_metric, best_metric_epoch
                )
            )
            
            torch.save(model.state_dict(), f"last_epoch_{encode_mod}.pth")
        
     #  lr decrease
#     lr_set.append(schedular.get_last_lr()[0])
#     schedular.step()
print(f"train completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")

# Reproduce

In [ ]:
model.load_state_dict(torch.load("EfficientV2.pth"))

## Validation

In [ ]:
model.eval()
dice_metric.reset()
with torch.no_grad():
    val_images = None
    val_labels = None
    val_outputs = None
#     show_val = True
    for val_data in val_loader:
        val_images, val_labels = val_data["img"].to(device), val_data["seg"].to(device)

        val_outputs, _ = model(val_images) #forward
        val_outputs = Resize([-1, 1716, 942])(val_outputs)
        val_outputs = [post_trans(i) for i in decollate_batch(val_outputs)]                
        
        visualize( 
            image=val_images[0].cpu().permute(1,2,0), 
            ground_truth_mask=val_labels[0].cpu().permute(1,2,0), 
            predicted_mask=val_outputs[0].cpu().permute(1,2,0)
        ) 
        
        # compute metric for current iteration
        dice_metric(y_pred=val_outputs, y=val_labels)
        print(dice_metric.aggregate())
    # aggregate the final mean dice result
    metric = dice_metric.aggregate().item()
    print("metric = ", metric)
    # reset the status for next validation round
    dice_metric.reset()

## Public Data

In [ ]:
tempdir = "./AICUP/Demo_Image/Public_Image/"
test_images = sorted(glob.glob(os.path.join(tempdir, "*.jpg")))


print(f" {len(test_images)} test_images")

test_files = [{"img": img} for img in test_images[:]]
test_files[:5]

In [ ]:
test_transforms = Compose(
    [
        LoadImaged(keys=["img"]),
        AsChannelFirstd(keys=["img"]),
        ScaleIntensityd(keys=["img"]),
        Resized(keys=["img"], spatial_size=[800, 800]),
        EnsureTyped(keys=["img"])
    ]
)
test_ds = monai.data.Dataset(data=test_files, transform=test_transforms)
test_loader = DataLoader(test_ds, batch_size=1,  collate_fn=list_data_collate)
alls = sorted(glob.glob(tempdir + "*.jpg"))

In [ ]:
model.eval()
with torch.no_grad():
    for i, test_data in enumerate(test_loader):
        test_images = test_data["img"].to(device)

        test_outputs, _ = model(test_images) #forward
        test_outputs = Resize([-1, 1716, 942])(test_outputs)
        
        saverPD = SaveImage(output_dir=f"./{encode_mod}/outputs1", output_ext=".png", output_postfix=f"{alls[i].split('/')[-1].split('.')[0]}",scale=255,separate_folder=False)
        saverPD(test_outputs[0].cpu())
        
        test_outputs = [post_trans(i) for i in decollate_batch(test_outputs)]
        

In [ ]:
alls = sorted(glob.glob(f"./{encode_mod}/outputs1/*.png"))
alls[:5]

In [ ]:
for jss in alls:
    os.rename(jss, os.path.join(*jss.split("/")[:-1], jss.split("/")[-1].split("_", 1)[-1]))

# Private Data

In [ ]:
tempdir = "./AICUP/Demo_Image/Private_Image/"
private_images = sorted(glob.glob(os.path.join(tempdir, "*.jpg")))


print(f" {len(private_images)} private_images")

private_files = [{"img": img} for img in private_images[:]]
private_files[:5]

In [ ]:
test_transforms = Compose(
    [
        LoadImaged(keys=["img"]),
        AsChannelFirstd(keys=["img"]),
        ScaleIntensityd(keys=["img"]),
        Resized(keys=["img"], spatial_size=[800, 800]),
        EnsureTyped(keys=["img"])
    ]
)
test_ds = monai.data.Dataset(data= private_files, transform=test_transforms)
test_loader = DataLoader(test_ds, batch_size=1,  collate_fn=list_data_collate)
alls = sorted(glob.glob(tempdir + "*.jpg"))

In [ ]:
model.eval()
steppp = 0
with torch.no_grad():
    for i, test_data in enumerate(test_loader):
        test_images = test_data["img"].to(device)
        test_outputs, _ = model(test_images) #forward
        test_outputs = Resize([-1, 1716, 942])(test_outputs)

        saverPD = SaveImage(output_dir=f"./{encode_mod}/outputsp", output_ext=".png", output_postfix=f"{alls[i].split('/')[-1].split('.')[0]}",scale=255,separate_folder=False)
        saverPD(test_outputs[0].cpu())
        
        test_outputs = [post_trans(i) for i in decollate_batch(test_outputs)]        
        imgg = Image.open(alls[i])
    
        if steppp%30 == 0:
            visualize( 
                image=test_images[0].cpu().permute(1,2,0), 
                ground_truth_images=imgg, 
                predicted_mask=test_outputs[0].squeeze().cpu().numpy().round()
            ) 
        steppp += 1
        

In [ ]:
alls = sorted(glob.glob(f"./{encode_mod}/outputsp/*.png"))
alls[:5]

In [ ]:
for jss in alls:
    os.rename(jss, os.path.join(*jss.split("/")[:-1], jss.split("/")[-1].split("_", 1)[-1]))

## Move

In [ ]:
!mv EfficientV2/outputsp/* EfficientV2/outputs1

# Zip

In [ ]:
!zip -r EfficientV2.zip EfficientV2